# 6. Majorana operators

The Majorana operators are defined in terms of the fermionic creation and annihilation operators for a quantum system with $L$ modes (or orbitals) as
\begin{align*}
m_{2j}   &= a_j^{\dagger} + a_j,\\
m_{2j+1} &= i (a_j^{\dagger} - a_j) \quad \text{for } j = 0, 1, \dots, L - 1.
\end{align*}
(There are various conventions in the literature, some of which include a factor $\frac{1}{2}$ or $\frac{1}{\sqrt{2}}$ in their definition.) Thus, there are $2 L$ Majorana operators in total. By definition, they are Hermitian $2^L \times 2^L$ matrices and satisfy the anti-commutation relations
\begin{equation*}
\{ m_j, m_k \} = 2 \delta_{j,k} \quad \text{for all } j, k = 0, \dots, 2 L - 1.
\end{equation*}
In particular, $m_j^2 = I$ (identity) for all $j = 0, \dots, 2 L - 1$.

By analogy with the quantum harmonic oscillator, one could regard them as position and momentum operators.

In [1]:
import fermi_relations as fr
import numpy as np
from scipy.linalg import expm, logm, schur, block_diag
from scipy import sparse
import scipy.sparse.linalg as spla

In [2]:
# number of modes (or orbitals) L
nmodes = 7

mlist = fr.construct_majorana_operators(nmodes)
print("len(mlist):", len(mlist))
print("mlist[0].shape:", mlist[0].shape)

# verify anti-commutation relations
err = 0
spid = sparse.identity(2**nmodes)
for j, mj in enumerate(mlist):
    for k, mk in enumerate(mlist):
        delta = (1 if j == k else 0)
        err += spla.norm(fr.anti_comm(mj, mk) - 2 * delta * spid)
    # mj^2 must be the identity operation
    err += spla.norm(mj @ mj - spid)
print("err:", err)

len(mlist): 14
mlist[0].shape: (128, 128)
err: 0.0


As a consequence of the anti-commutation relations, the Majorana operators (rescaled by $2^{-L/2}$) form an orthonormal basis of operators on the fermionic Fock space with respect to the inner product
\begin{equation*}
\langle A, B \rangle = \mathrm{tr}[A^{\dagger} B].
\end{equation*}

In [3]:
# numerically verify orthonormality property
inner_products = np.zeros((2*nmodes, 2*nmodes), dtype=complex)
for j, mj in enumerate(mlist):
    for k, mk in enumerate(mlist):
        inner_products[j, k] = (mj @ mk).trace() / 2**nmodes
# overlap matrix should be the identity
np.linalg.norm(inner_products - np.identity(2*nmodes))

np.float64(0.0)

Re-expressing the fermionic creation and annihilation operators in terms of Majorana operators,
\begin{align*}
a_j^{\dagger} &= \frac{1}{2} (m_{2j} - i m_{2j + 1}), \\
a_j           &= \frac{1}{2} (m_{2j} + i m_{2j + 1}) \quad \text{for } j = 0, \dots, L - 1
\end{align*}
allows switching between the two representations.

We define the Majorana operator $m_{\varphi}$ for a given state $\varphi \in \mathbb{R}^{2L}$ as:
\begin{equation*}
m_{\varphi} = \sum_{j=0}^{2L-1} \varphi_j m_j.
\end{equation*}

The following derivations are based on the publication (Adrian Chapman and Steven T. Flammia, Characterization of solvable spin models via graph invariants, [Quantum 4, 278 (2020)](https://doi.org/10.22331/q-2020-06-04-278)). 

Since fermionic and Majorana operators are linear combinations of each other, a free-fermion model Hamiltonian can be expressed in the form
\begin{equation*}
H = i \sum_{j,k=0}^{2L-1} h_{jk} m_j m_k
\end{equation*}
with $h \in \mathbb{R}^{2L \times 2L}$. The prefactor $i$ is chosen by convention such that the coefficients $h_{jk}$ are real-valued if $H$ is Hermitian. Because of the anti-commutation property of the Majorana operators, we take $h$ to be anti-symmetric without loss of generality, i.e., $h^T = -h$. As before, the time evolution governed by $H$ is the unitary
\begin{equation*}
U = e^{-i H}.
\end{equation*}
To simplify the notation, we have absorbed the time variable into $H$.

Analogous to notebook 2, we can derive the following unitary time evolution transformation property of the Majorana operators for any $\varphi \in \mathbb{R}^{2L}$:
\begin{equation*}
U^{\dagger} m_{\varphi} U = m_{u^T \varphi}, \quad u = e^{4 h},
\end{equation*}
where $u^T \varphi$ is the usual matrix-vector product and $u^T$ is the transpose of $u$ (which is equal to its inverse since $u$ is orthogonal).

As before, this relation follows from the Campbell identity
\begin{equation*}
\mathrm{Ad}_{e^{i H}} = e^{\mathrm{ad}_{i H}}
\end{equation*}
where $\mathrm{Ad}_{e^X} Y = e^X Y e^{-X}$, $\mathrm{ad}_X Y = [X, Y]$, and
\begin{equation*}
e^{\mathrm{ad}_X} Y = Y + [X, Y] + \frac{1}{2!} [X, [X, Y]] + \dots
\end{equation*}
together with
\begin{equation*}
\big[i H, m_{\ell}\big] = \Bigg[-\sum_{j,k=0}^{2L-1} h_{jk} m_j m_k, m_{\ell}\Bigg] = 4 \sum_{k=0}^{2L-1} h_{\ell k} m_k.
\end{equation*}

In [4]:
# random number generator
rng = np.random.default_rng(47)

In [5]:
# random real anti-symmetric single-particle Hamiltonian
h = fr.antisymmetrize(rng.standard_normal((2*nmodes, 2*nmodes)))

# Hamiltonian on whole Fock space, Eq. (11) in Chapman and Flammia paper
mlist = fr.construct_majorana_operators(nmodes)
hfock = 1j * sum(h[j, k] * (mlist[j] @ mlist[k])
                    for j in range(2*nmodes)
                    for k in range(2*nmodes))
# must be Hermitian
err = spla.norm(hfock.conj().T - hfock)

ufock = expm(-1j*hfock.toarray())
# must be unitary
err += np.linalg.norm(ufock.conj().T @ ufock - np.identity(ufock.shape[1]))

# Majorana mode transformation, Eq. (12) in Chapman and Flammia paper
u = expm(4*h)
for j, mj in enumerate(mlist):
    err += np.linalg.norm(ufock.conj().T @ mj @ ufock
                          - sum(u[j, k] * mlist[k].toarray() for k in range(len(mlist))))

# alternative construction
# define a random "orbital" state
phi = rng.standard_normal(2*nmodes)
phi /= np.linalg.norm(phi)
# numerically verify the equation above
err += np.linalg.norm(ufock.conj().T @ fr.orbital_majorana_op(phi) @ ufock
                      - fr.orbital_majorana_op(u.T @ phi))

print("err:", err)

err: 6.290998950061377e-13


Similar to notebook 4, we can diagonalize the Hamiltonian, as follows. Since $h$ is real anti-symmetric, it can be block-diagonalized by a matrix $w \in \text{SO}(2L, \mathbb{R})$ (special orthogonal group):
\begin{equation*}
h = w \begin{pmatrix} 0 & -\epsilon_0 \\ \epsilon_0 & 0 \\ & & 0 & -\epsilon_1 \\ & & \epsilon_1 & 0 \\ & & & & \ddots \\ & & & & & 0 & -\epsilon_{L-1} \\ & & & & & \epsilon_{L-1} & 0 \end{pmatrix} w^T,
\end{equation*}
with real "energy" eigenvalues $\epsilon_0, \dots, \epsilon_{L-1}$ of $h$.

In [6]:
# block-diagonalize 'h', Eq. (14) in Chapman and Flammia paper
t, w = schur(h, output="real")
if np.linalg.det(w) < 0:
    w[:, 0] = -w[:, 0]
    t[:, 0] = -t[:, 0]
    t[:, 1] = -t[:, 1]
# 'w' must be in SO(2*nmodes, R)
assert np.isrealobj(w)
assert np.allclose(w.T @ w, np.identity(w.shape[1]))
assert np.allclose(np.linalg.det(w), 1)
print("np.linalg.det(w):", np.linalg.det(w))

# 'w' must diagonalize 'h'
assert np.allclose(h @ w, w @ t)

# verify block-diagonal structure
blocks = [t[i:i+2, i:i+2] for i in range(0, 2*nmodes, 2)]
assert np.allclose(t, block_diag(*blocks))
en_list = np.array([b[1, 0] for b in blocks])
assert len(en_list) == nmodes
for en, b in zip(en_list, blocks):
    assert np.allclose(b, [[0, -en], [en, 0]])
print("en_list:", en_list)

np.linalg.det(w): 1.0000000000000033
en_list: [ 3.85241591  3.0882257   2.83013621  2.11236519  1.38476872 -0.06198893
  0.63644431]


Inserting the block-diagonalized form of $h$ into the definition of $H$ leads to
\begin{equation*}
\begin{split}
H &= i \sum_{j,k=0}^{2L-1} \sum_{\ell=0}^{L-1} \epsilon_{\ell} (w_{j,2\ell+1} w_{k,2\ell} - w_{j,2\ell} w_{k,2\ell+1}) m_j m_k \\
&= i \sum_{\ell=0}^{L-1} \epsilon_{\ell} \left( \left(\sum_{j=0}^{2L-1} w_{j,2\ell+1} m_j\right) \left(\sum_{k=0}^{2L-1} w_{k,2\ell} m_k\right) - \left(\sum_{j=0}^{2L-1} w_{j,2\ell} m_j\right) \left(\sum_{k=0}^{2L-1} w_{k,2\ell+1} m_k\right) \right) \\
&= i \sum_{\ell=0}^{L-1} \epsilon_{\ell} \left(\tilde{m}_{2\ell+1} \tilde{m}_{2\ell} - \tilde{m}_{2\ell} \tilde{m}_{2\ell+1}\right) \\
&= -2 i \sum_{\ell=0}^{L-1} \epsilon_{\ell} \tilde{m}_{2\ell} \tilde{m}_{2\ell+1},
\end{split}
\end{equation*}
where we have defined the "rotated" Majorana operators as $\tilde{m}_{\ell} = \sum_{j=0}^{2L-1} w_{j\ell} m_j$.

In [7]:
# represent H in terms of the rotated Majorana operators
hfock_alt = -2j * sum(en_list[i] * fr.orbital_majorana_op(w[:, 2*i]) @ fr.orbital_majorana_op(w[:, 2*i+1])
                       for i in range(nmodes))
# compare (difference should be numerically zero)
spla.norm(hfock_alt - hfock)

np.float64(1.9718431013121616e-13)

If we want to find the corresponding basis change matrix on the whole Fock space which diagonalizes $H$, we must first represent $w$ as a matrix exponential, i.e.,
\begin{equation*}
w = e^{4 g}
\end{equation*}
with $g \in \mathbb{R}^{2L \times 2L}$ anti-symmetric:

In [8]:
# Eq. (15) in Chapman and Flammia paper
g = logm(w) / 4
assert np.isrealobj(g)
print("g.shape:", g.shape)
# must be anti-symmetric
print(np.linalg.norm(g.T + g))
print(np.linalg.norm(w - expm(4*g)))

g.shape: (14, 14)
5.497315937482174e-15
1.0326385362050844e-14


As for the Hamiltonian above, we define a Hermitian operator on the whole Fock space via the coefficient matrix $g$:
\begin{equation*}
G = i \sum_{j,k=0}^{2L-1} g_{jk} m_j m_k.
\end{equation*}
The matrix exponential of $G$ is then the sought change of basis matrix:
\begin{equation*}
W = e^{-i G}
\end{equation*}
block-diagonalizes $H$:
\begin{equation*}
\begin{split}
W^{\dagger} H W
&= i \sum_{j,k=0}^{2L-1} h_{jk} \left(W^{\dagger} m_j W\right) \left(W^{\dagger} m_k W\right)
 = i \sum_{j,k=0}^{2L-1} h_{jk} m_{w_{j,:}} m_{w_{k,:}} \\
&= i \sum_{j,k=0}^{2L-1} \sum_{j',k'=0}^{2L-1} w_{j j'} h_{jk} w_{k k'} m_{j'} m_{k'}
 = i \sum_{j',k'=0}^{2L-1} \left(w^T h w\right)_{j'k'} m_{j'} m_{k'} \\
&= i \sum_{\ell=0}^{L-1} \epsilon_{\ell} \left(m_{2\ell+1} m_{2\ell} - m_{2\ell} m_{2\ell+1}\right) \\
&= -2 i \sum_{\ell=0}^{L-1} \epsilon_{\ell} m_{2\ell} m_{2\ell+1}.
\end{split}
\end{equation*}
Note that this is essentially the same derivation as above (using the rotated Majorana operators). We verify the identities numerically:

In [9]:
gfock = 1j * sum(g[i, j] * (mlist[i] @ mlist[j])
                    for i in range(2*nmodes)
                    for j in range(2*nmodes))
# must be Hermitian
err = spla.norm(gfock.conj().T - gfock)

wfock = expm(-1j*gfock.toarray())
# must be unitary
err += np.linalg.norm(wfock.conj().T @ wfock - np.identity(wfock.shape[1]))

# W block-diagonalizes H, see derivation above and Eq. (17) in Chapman and Flammia paper
hfock_diag = -2j * sum(en_list[i] * (mlist[2*i] @ mlist[2*i+1]) for i in range(nmodes))
err += np.linalg.norm(wfock.conj().T @ hfock @ wfock - hfock_diag.toarray())

print("err:", err)

err: 7.139437070979179e-13
